In [2]:
import numpy as np
import pandas as pd
import allel
import random

In [6]:
#### params 
# get the option of pi beta and alle freq chosen 
chr_number = 5
n_ecotypes = 225 
vcf_file = '../data/chr5_grenenet.vcf'

#pi_option =  snakemake.params['pi']
#beta = snakemake.params['beta']
#print(beta)
#alelle_freq_option = snakemake.params['allele_freq']

#get the actual values
n_optima = 15

pi = 0.000005

allele_freq = [0.05, 0.5]
lower_bound = float(allele_freq[0])
upper_bound = float(allele_freq[1])

#bed_sc = snakemake.output["bed_sc"]
#optima_slim = snakemake.output["slim"]
######

## get vcf file
vcf_og = allel.read_vcf(vcf_file)




In [10]:
def scale(beta_values):
    min_value = np.min(beta_values)
    max_value = np.max(beta_values)
    beta_values = ((beta_values - min_value) / (max_value - min_value)) * (5 - 1) + 1
    return beta_values

In [ ]:


def getting_selection_coef(vcf_og, pi, beta, lower_bound, upper_bound):
    ### calculation of the number of contributing positions and effect sizes of each ###################
    pos = vcf_og['variants/POS']
    ## get all the pos from the vcf file
    n_pos = len(pos)
    geno_og = vcf_og["calldata/GT"]
    ## freq of alternative alleles
    alt_al_count = geno_og.sum(axis=2).sum(axis=1)
    n_pos_contr = round(pi*n_pos)
    
    alelle_dist = pd.DataFrame({'alt_al_count':alt_al_count, 'pos':pos})

    alelle_dist['alt_al_freq'] = alelle_dist['alt_al_count'] / (n_ecotypes*2)

    if beta == 'betaprop': 
        alelle_dist = alelle_dist.copy()
        beta_values = alelle_dist.loc[:,('alt_al_freq')] * 5
        beta_values = scale(beta_values)
        alelle_dist.loc[:,'beta'] = beta_values

    elif beta == 'betanonprop':
        alelle_dist = alelle_dist.copy()
        beta_values = 5 - (alelle_dist.loc[:,('alt_al_freq')] * 5 )
        beta_values = scale(beta_values)
        alelle_dist.loc[:,'beta'] = beta_values

    alelle_dist = alelle_dist[(alelle_dist['alt_al_freq'] > lower_bound) & (alelle_dist['alt_al_freq'] <= upper_bound)]

    pos_contr = alelle_dist['pos'].sample(n_pos_contr)

    alelle_dist = alelle_dist[alelle_dist['pos'].isin(pos_contr)]

    effect_sizes = [np.random.normal(0, beta, 1) for beta in alelle_dist['beta']]
    effect_sizes = effect_sizes.copy()

    alelle_dist.loc[:,'effect_sizes'] = np.ravel(effect_sizes).copy()
    alelle_dist.loc[:,'effect_sizes'] = alelle_dist.loc[:,'effect_sizes'].round(4)
    effect_sizes = np.round(alelle_dist['effect_sizes'], 4)

    pos = pd.Series(pos)
    pos.name = 'pos'
    bed_file = pd.merge(pos, alelle_dist[['pos', 'effect_sizes']], on='pos', how='outer').fillna(0)
    bed_file['positions_from'] = bed_file['pos'] - 1
    bed_file['chr'] = chr_number
    bed_file = bed_file[['chr', 'positions_from', 'pos', 'effect_sizes']]

    return bed_file

def calc_optima(bed_file, vcf_og):
    geno_og = vcf_og["calldata/GT"]
    alt_al_per_pos = geno_og.sum(axis=2) #.sum(axis=1)
    phenotypes = []

    for i in range(alt_al_per_pos.shape[1]):
        gen_effectsize = np.multiply(alt_al_per_pos[:, i] , np.array(bed_file['effect_sizes']))
        phenotypes.append(gen_effectsize.sum())

    max_pheno = max(phenotypes)

    min_pheno = min(phenotypes)

    optima = np.random.uniform(min_pheno, max_pheno, n_optima - 2)
    optima = np.append(optima, [min_pheno, max_pheno])
    return optima

bed_file = getting_selection_coef(vcf_og, pi, beta, lower_bound, upper_bound)
bed_file.to_csv(bed_sc, sep='\t', header=False, index=False) # selection_coef_chr5.bed

pheno_optima = calc_optima(bed_file, vcf_og)
pd.Series(pheno_optima).to_csv(optima_slim, header=None, sep='\n', index=None)

In [8]:
beta = 'betanonprop'

In [13]:
### calculation of the number of contributing positions and effect sizes of each ###################
pos = vcf_og['variants/POS']
## get all the pos from the vcf file
n_pos = len(pos)
geno_og = vcf_og["calldata/GT"]

In [17]:
geno_og

array([[[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       ...,

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]]], dtype=int8)

In [18]:

## freq of alternative alleles
alt_al_count = geno_og.sum(axis=2).sum(axis=1)
n_pos_contr = round(0.1*n_pos)

alelle_dist = pd.DataFrame({'alt_al_count':alt_al_count, 'pos':pos})

alelle_dist['alt_al_freq'] = alelle_dist['alt_al_count'] / (n_ecotypes*2)

if beta == 'betaprop': 
    alelle_dist = alelle_dist.copy()
    beta_values = alelle_dist.loc[:,('alt_al_freq')] * 5
    beta_values = scale(beta_values)
    alelle_dist.loc[:,'beta'] = beta_values

elif beta == 'betanonprop':
    alelle_dist = alelle_dist.copy()
    beta_values = 5 - (alelle_dist.loc[:,('alt_al_freq')] * 5 )
    beta_values = scale(beta_values)
    alelle_dist.loc[:,'beta'] = beta_values

alelle_dist = alelle_dist[(alelle_dist['alt_al_freq'] > lower_bound) & (alelle_dist['alt_al_freq'] <= upper_bound)]

pos_contr = alelle_dist['pos'].sample(n_pos_contr)

alelle_dist = alelle_dist[alelle_dist['pos'].isin(pos_contr)]

effect_sizes = [np.random.normal(0, beta, 1) for beta in alelle_dist['beta']]
effect_sizes = effect_sizes.copy()

alelle_dist.loc[:,'effect_sizes'] = np.ravel(effect_sizes).copy()
alelle_dist.loc[:,'effect_sizes'] = alelle_dist.loc[:,'effect_sizes'].round(4)
effect_sizes = np.round(alelle_dist['effect_sizes'], 4)

pos = pd.Series(pos)
pos.name = 'pos'
bed_file = pd.merge(pos, alelle_dist[['pos', 'effect_sizes']], on='pos', how='outer').fillna(0)
bed_file['positions_from'] = bed_file['pos'] - 1
bed_file['chr'] = chr_number
bed_file = bed_file[['chr', 'positions_from', 'pos', 'effect_sizes']]

In [20]:
alelle_dist

,alt_al_count,pos,alt_al_freq,beta,effect_sizes
4,32,620146,0.071111,4.962264,-2.1544
27,80,5415374,0.177778,4.905660,8.3796
29,128,5571785,0.284444,4.849057,-0.4071
33,32,6346978,0.071111,4.962264,6.3330
34,64,6733596,0.142222,4.924528,-7.7236
56,192,10015566,0.426667,4.773585,5.2770
70,80,12193845,0.177778,4.905660,1.7751
76,200,12971731,0.444444,4.764151,-7.6727
86,32,14211701,0.071111,4.962264,-3.6997
90,208,15927575,0.462222,4.754717,0.7993


In [19]:
effect_sizes

4      -2.1544
27      8.3796
29     -0.4071
33      6.3330
34     -7.7236
56      5.2770
70      1.7751
76     -7.6727
86     -3.6997
90      0.7993
93      1.8032
119    -6.1280
124     7.5341
127    -5.5198
150    11.4958
153    -4.2216
167    -2.5924
172    -3.6470
174    -8.4994
175     2.1708
Name: effect_sizes, dtype: float64

In [23]:
[abs(np.random.normal(0, beta, 1)) for beta in alelle_dist['beta']]

[array([2.07706234]),
 array([4.18162726]),
 array([4.16041672]),
 array([0.67296217]),
 array([5.20508181]),
 array([1.0418717]),
 array([4.11958659]),
 array([0.06750712]),
 array([1.33297966]),
 array([2.79536142]),
 array([1.17411296]),
 array([2.08406651]),
 array([3.04128033]),
 array([3.05312375]),
 array([3.90863213]),
 array([5.07889731]),
 array([2.4624949]),
 array([2.95655842]),
 array([7.43460793]),
 array([0.59851202])]